# Analysis of gaming ranking vs price move

In [25]:
!pip install gamedatacrunch
!pip install steamspypi
import steamspypi
from pygooglenews import GoogleNews
from bs4 import BeautifulSoup
from langdetect import detect
import numpy as np 
from datetime import datetime, timedelta
import yfinance as yf
import matplotlib.pyplot as plt
from IPython.display import display, DisplayObject
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import requests
import pandas as pd
import time, calendar
import os

In [4]:
def get_stock_data_by_interval(symbol, start_datetime, end_datetime, interval='1m'):
    # Interval argument examples : '1m', '1h', '1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max'
    # Start and end datetime format example : '2000-01-01 12:34:00'
    start_timestamp = calendar.timegm(time.strptime(start_datetime, '%Y-%m-%d %H:%M:%S')) - 86400
    end_timestamp = calendar.timegm(time.strptime(end_datetime, '%Y-%m-%d %H:%M:%S')) + 86400
    
    url = f'https://query1.finance.yahoo.com/v8/finance/chart/{symbol}?interval={interval}&period1={start_timestamp}&period2={end_timestamp}'
    result = requests.get(url)
    
    data = result.json()
    body = data['chart']['result'][0]
    price_data_body = body['indicators']['quote'][0]
    
    df = pd.DataFrame()
    df['Datetime'] = body['timestamp']
    df['Datetime'] = pd.to_datetime(df['Datetime'],unit='s')
    df['high'] = price_data_body['high']
    df['low'] = price_data_body['low']
    df['volume'] = price_data_body['volume']
    df['open'] = price_data_body['open']
    df['close'] = price_data_body['close']
    
    return df

In [7]:
get_stock_data_by_interval(symbol='TTWO', start_datetime='1996-01-01 12:34:00', end_datetime='2021-02-20 23:00:00', interval='1d')

,Datetime,high,low,volume,open,close
0,1997-04-15 13:30:00,3.916667,3.416667,1785600,3.666667,3.916667
1,1997-04-16 13:30:00,4.166667,3.583333,443700,3.916667,3.708333
2,1997-04-17 13:30:00,4.083333,3.750000,252150,3.750000,3.833333
3,1997-04-18 13:30:00,4.000000,3.833333,165900,3.833333,4.000000
4,1997-04-21 13:30:00,4.000000,3.916667,77400,3.916667,4.000000
...,...,...,...,...,...,...
5998,2021-02-12 14:30:00,200.279999,194.720001,1264000,196.500000,199.860001
5999,2021-02-16 14:30:00,201.000000,194.449997,1532500,200.729996,195.789993
6000,2021-02-17 14:30:00,196.169998,191.100006,1066800,194.630005,195.929993
6001,2021-02-18 14:30:00,196.770004,192.559998,808700,192.639999,196.229996


In [27]:
data_request = dict()

data_request['request'] = 'name'
data_request['name'] = 'Counter-Strike: Global Offensive'

data = steamspypi.download(data_request)
data

Requested name is not standard.
No request field could be found.
Incorrect request: download is cancelled.


{}

In [26]:
import gamedatacrunch as gdc

data = gdc.load_as_steam_api()
data

{'applist': {'apps': [{'appid': 730,
    'name': 'Counter-Strike: Global Offensive'},
   {'appid': 1091500, 'name': 'Cyberpunk 2077'},
   {'appid': 255710, 'name': 'Cities: Skylines'},
   {'appid': 10, 'name': 'Counter-Strike'},
   {'appid': 240, 'name': 'Counter-Strike: Source'},
   {'appid': 268910, 'name': 'Cuphead'},
   {'appid': 203770, 'name': 'Crusader Kings II'},
   {'appid': 204360, 'name': 'Castle Crashers®'},
   {'appid': 363970, 'name': 'Clicker Heroes'},
   {'appid': 231430, 'name': 'Company of Heroes 2'},
   {'appid': 219640, 'name': 'Chivalry: Medieval Warfare'},
   {'appid': 440900, 'name': 'Conan Exiles'},
   {'appid': 311210, 'name': 'Call of Duty®: Black Ops III'},
   {'appid': 386180, 'name': 'Crossout'},
   {'appid': 273110, 'name': 'Counter-Strike Nexon: Studio'},
   {'appid': 1158310, 'name': 'Crusader Kings III'},
   {'appid': 884660, 'name': 'CRSED: F.O.A.D.'},
   {'appid': 223710, 'name': 'Cry of Fear'},
   {'appid': 645630, 'name': 'Car Mechanic Simulator 201